### **Reconhecedor de padrões baseado em LZW**

#### Pré-processamento das imagens

##### 1. Retirada do cabeçalho dos dados

In [1]:
import os

In [2]:
# Diretório de origem
diretorio_origem = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces"

# Diretório de destino
diretorio_destino = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_sc"

In [3]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino
    caminho_destino_subpasta = os.path.join(diretorio_destino, subpasta)
    os.makedirs(caminho_destino_subpasta, exist_ok=True)

    # Percorre os arquivos de imagem na subpasta atual
    for arquivo in os.listdir(caminho_subpasta):
        if arquivo.endswith(".pgm"):
            caminho_arquivo_origem = os.path.join(caminho_subpasta, arquivo)

            # Abre o arquivo de origem em modo binário
            with open(caminho_arquivo_origem, "rb") as arquivo_origem:
                dados = arquivo_origem.read()

            # Remove as três primeiras linhas
            novos_dados = dados.split(b"\n", 3)[-1]

            # Cria o caminho do arquivo de destino
            caminho_arquivo_destino = os.path.join(caminho_destino_subpasta, arquivo)

            # Salva os novos dados no arquivo de destino em modo binário
            with open(caminho_arquivo_destino, "wb") as arquivo_destino:
                arquivo_destino.write(novos_dados)

#### Geração dos modelos

##### 1. Divisão aleatória do banco entre instâncias de "treinamento" e "teste" - **ocorre a cada novo valor de k**

In [4]:
import os
import shutil
import random

In [5]:
# Diretório de origem
diretorio_origem = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_sc"

# Diretório de destino para treinamento
diretorio_destino_treinamento = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_div\train"

# Diretório de destino para teste
diretorio_destino_teste = r"C:\Users\franklin.coelho\Documents\encode-decode-lzw\db\orl_faces_div\test"

# Porcentagem de imagens para treinamento (90%)
porcentagem_treinamento = 0.9

In [6]:
# Exclui os diretórios de destino, se já existirem
if os.path.exists(diretorio_destino_treinamento):
    shutil.rmtree(diretorio_destino_treinamento)
if os.path.exists(diretorio_destino_teste):
    shutil.rmtree(diretorio_destino_teste)

In [7]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta_origem = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino para treinamento
    caminho_subpasta_destino_treinamento = os.path.join(diretorio_destino_treinamento, subpasta)
    os.makedirs(caminho_subpasta_destino_treinamento)

    # Cria a subpasta correspondente no diretório de destino para teste
    caminho_subpasta_destino_teste = os.path.join(diretorio_destino_teste, subpasta)
    os.makedirs(caminho_subpasta_destino_teste)

    # Lista os arquivos na subpasta atual
    arquivos = os.listdir(caminho_subpasta_origem)
    random.shuffle(arquivos)  # Embaralha a ordem dos arquivos

    # Calcula o número de arquivos para treinamento e teste
    num_arquivos_treinamento = int(len(arquivos) * porcentagem_treinamento)
    num_arquivos_teste = len(arquivos) - num_arquivos_treinamento

    # Divide os arquivos entre treinamento e teste
    arquivos_treinamento = arquivos[:num_arquivos_treinamento]
    arquivos_teste = arquivos[num_arquivos_treinamento:]

    # Move os arquivos para as respectivas pastas de treinamento e teste
    for arquivo in arquivos_treinamento:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_treinamento, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

    for arquivo in arquivos_teste:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_teste, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

##### 2. Construção dos modelos/dicionários para cada classe

In [126]:
dict_size = 256

In [127]:
def lzw_encode(input_file, dictionary, max_dict_size):
    current_sequence = bytes()
    global dict_size

    with open(input_file, "rb") as file:
        while True:
            symbol = file.read(1)
            if not symbol:
                break

            next_sequence = current_sequence + symbol
            if next_sequence in dictionary:
                current_sequence = next_sequence
            else:
                if dict_size < 2 ** max_dict_size:
                    dictionary[next_sequence] = dict_size
                    dict_size += 1
                current_sequence = symbol

In [129]:
def encode_images_in_folders(folder_path, max_dict_size):
    dictionaries = []

    # Verifica se o tamanho máximo do dicionário está dentro do intervalo válido
    if not (9 <= max_dict_size <= 16):
        raise ValueError("O tamanho máximo do dicionário deve estar entre 9 e 16.")

    # Percorre todas as pastas
    for folder_name in os.listdir(folder_path):
        folder_dir = os.path.join(folder_path, folder_name)
        dictionary = {bytes([i]): i for i in range(256)}

        # Codifica as imagens na pasta atual
        for image_name in os.listdir(folder_dir):
            image_path = os.path.join(folder_dir, image_name)

            # Realiza a codificação LZW para cada imagem
            lzw_encode(image_path, dictionary, max_dict_size)
        
        global dict_size
        dict_size = 256

        # Adiciona o dicionário da pasta à lista de dicionários
        dictionaries.append((folder_name, dictionary))

    return dictionaries

In [138]:
# Diretório das pastas de imagens
folder_path = diretorio_destino_treinamento
# Tamanho máximo do dicionário (2^k)
max_dict_size = 9
# Codifica as imagens em todas as pastas e obtém a lista de dicionários
all_dictionaries = encode_images_in_folders(folder_path, max_dict_size)

In [139]:
folder_name, dictionary = all_dictionaries[8]
print(f"Dicionário da pasta '{folder_name}':")
print(dictionary)
print(f"Tamanho: {len(dictionary)}")

Dicionário da pasta 's17':
{b'\x00': 0, b'\x01': 1, b'\x02': 2, b'\x03': 3, b'\x04': 4, b'\x05': 5, b'\x06': 6, b'\x07': 7, b'\x08': 8, b'\t': 9, b'\n': 10, b'\x0b': 11, b'\x0c': 12, b'\r': 13, b'\x0e': 14, b'\x0f': 15, b'\x10': 16, b'\x11': 17, b'\x12': 18, b'\x13': 19, b'\x14': 20, b'\x15': 21, b'\x16': 22, b'\x17': 23, b'\x18': 24, b'\x19': 25, b'\x1a': 26, b'\x1b': 27, b'\x1c': 28, b'\x1d': 29, b'\x1e': 30, b'\x1f': 31, b' ': 32, b'!': 33, b'"': 34, b'#': 35, b'$': 36, b'%': 37, b'&': 38, b"'": 39, b'(': 40, b')': 41, b'*': 42, b'+': 43, b',': 44, b'-': 45, b'.': 46, b'/': 47, b'0': 48, b'1': 49, b'2': 50, b'3': 51, b'4': 52, b'5': 53, b'6': 54, b'7': 55, b'8': 56, b'9': 57, b':': 58, b';': 59, b'<': 60, b'=': 61, b'>': 62, b'?': 63, b'@': 64, b'A': 65, b'B': 66, b'C': 67, b'D': 68, b'E': 69, b'F': 70, b'G': 71, b'H': 72, b'I': 73, b'J': 74, b'K': 75, b'L': 76, b'M': 77, b'N': 78, b'O': 79, b'P': 80, b'Q': 81, b'R': 82, b'S': 83, b'T': 84, b'U': 85, b'V': 86, b'W': 87, b'X': 88, b'